# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    # Initialize, create an simple CNN with 2 conv layers, 1 maxpooling layer and 2 full-connected layer
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(stride=2,kernel_size=2))
        
        self.dense = nn.Sequential(
            nn.Linear(14*14*128,1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024,10))
        
    # Modify forward propagation function    
    def forward(self, x):
        x = self.conv1(x)
        x = x.view(-1, 14*14*128)
        x = self.dense(x)
        return x
    
model = SimpleNet()
if torch.cuda.is_available():
    model.cuda()
    
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    running_correct = 0
    print("Epoch{}/{}".format(epoch, NUM_EPOCHS))
    print('-'*10)
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images, labels = Variable(images), Variable(labels)
        if torch.cuda.is_available():
            images, labels = Variable(images.cuda()), Variable(labels.cuda())
        else:
            images, labels = Variable(images), Variable(labels)
        outputs = model(images)
        _,pred = torch.max(outputs.data, 1)
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        running_loss += loss.data.item()
        running_correct += torch.sum(pred == labels.data).item()   
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    testing_correct = 0
    for images, labels in tqdm(test_loader):
        if torch.cuda.is_available():
            images, labels = Variable(images.cuda(),requires_grad = False), Variable(labels.cuda(),requires_grad = False)
        else:
            images, labels = Variable(images,requires_grad = False), Variable(labels,requires_grad = False)
        outputs = model(images)
        _,pred = torch.max(outputs.data, 1)
        testing_correct += torch.sum(pred == labels.data).item()   
    print('Loss is:{:.4f}, Train Accuracy is:{:.4f}%, Test Accuracy is:{:.4f}%'
          .format(running_loss/len(train_dataset),100*running_correct/float(len(train_dataset)),100*testing_correct/float(len(test_dataset))))


Epoch0/10
----------


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:05<00:00, 25.02it/s]


Loss is:0.0014, Train Accuracy is:94.7250%, Test Accuracy is:97.9700%
Epoch1/10
----------


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:05<00:00, 13.62it/s]


Loss is:0.0004, Train Accuracy is:98.3433%, Test Accuracy is:98.5400%
Epoch2/10
----------


...

Loss is:0.0001, Train Accuracy is:99.4633%, Test Accuracy is:98.5600%
Epoch8/10
----------


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:03<00:00, 25.05it/s]


Loss is:0.0001, Train Accuracy is:99.4833%, Test Accuracy is:98.6100%
Epoch9/10
----------


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:05<00:00, 23.85it/s]


Loss is:0.0001, Train Accuracy is:99.5700%, Test Accuracy is:98.7700%


#### Q5:
Please print the training and testing accuracy.

In [5]:
print('Training accuracy: %0.4f%%' % (running_correct/len(train_dataset)*100))
print('Testing accuracy: %0.4f%%' % (testing_correct/len(test_dataset)*100))

Training accuracy: 99.5700%
Testing accuracy: 98.7700%
